In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib notebook
%reload_ext line_profiler

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import named_arrays as na
import named_arrays._vectors.vector_array_functions
import named_arrays._scalars.scalar_array_functions
import named_arrays._core_array_functions
import optika
import esis

In [ ]:

# Load the measured reflectivity of the witness sample
reflectivity_measured = esis.flights.f1.optics.primaries.materials.reflectivity_witness()

In [5]:

# Fit a multilayer stack to the measured reflectivity
# %lprun -f na.AbstractVectorArray.__array_matmul__ multilayer = esis.flights.f1.optics.primaries.materials.multilayer_witness()
# %lprun -f optika.materials.matrices.refraction multilayer = esis.flights.f1.optics.primaries.materials.multilayer_witness()
multilayer = esis.flights.f1.optics.primaries.materials.multilayer_witness()



       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 0.015342699917992492
             x: [ 2.139e+01  3.420e+00  6.978e-05  1.392e-05]
           nit: 330
          nfev: 550
 final_simplex: (array([[ 2.139e+01,  3.420e+00,  6.978e-05,  1.392e-05],
                       [ 2.139e+01,  3.420e+00,  1.545e-04,  0.000e+00],
                       ...,
                       [ 2.139e+01,  3.420e+00,  1.528e-04,  0.000e+00],
                       [ 2.139e+01,  3.420e+00,  0.000e+00,  7.460e-05]]), array([ 1.534e-02,  1.534e-02,  1.534e-02,  1.534e-02,
                        1.534e-02]))


In [ ]:
# Define the rays incident on the multilayer stack that will be used to
# compute the reflectivity
rays = optika.rays.RayVectorArray(
    wavelength=reflectivity_measured.inputs.wavelength,
    direction=na.Cartesian3dVectorArray(
        x=np.sin(reflectivity_measured.inputs.direction),
        y=0,
        z=np.cos(reflectivity_measured.inputs.direction),
    ),
)

# Compute the reflectivity of the fitted multilayer stack
reflectivity_fit = multilayer.efficiency(
    rays=rays,
    normal=na.Cartesian3dVectorArray(0, 0, -1),
)

# Plot the fitted vs. measured reflectivity
fig, ax = plt.subplots(constrained_layout=True)
na.plt.scatter(
    reflectivity_measured.inputs.wavelength,
    reflectivity_measured.outputs,
    ax=ax,
    label="measured"
);
na.plt.plot(
    rays.wavelength,
    reflectivity_fit,
    ax=ax,
    label="fitted",
    color="tab:orange",
);
ax.set_xlabel(f"wavelength ({rays.wavelength.unit:latex_inline})")
ax.set_ylabel("reflectivity")
ax.legend();

# Print the fitted multilayer stack
multilayer

In [3]:
x = na.linspace(0, 1, axis="x", num=100001)
y = na.linspace(0, 1, axis="x", num=100001)
z = na.linspace(0, 1, axis="x", num=100001)
a = na.Cartesian3dVectorArray(x, y, z)
b = na.Cartesian3dVectorArray(x, x, x)

In [203]:
def test():
    c = x + y * z
    # c = a @ b
    # d = a.x * b.x + a.y * b.y + a.z * b.z
    return c

# %lprun -f test test()
# %lprun -f na.AbstractCartesianVectorArray.__array_ufunc__ test()
# %lprun -f na.AbstractVectorArray.__array_ufunc__ test()
# %lprun -f na.AbstractScalarArray.__array_ufunc__ test()
%lprun -f na.ScalarArray.__post_init__ test()
# %lprun -f na.AbstractScalarArray.ndarray_aligned test()
# %lprun -f na.ScalarArray.shape.fget test()
# %lprun -f na.shape_broadcasted test()
# %lprun -f na.broadcast_shapes test()
# %lprun -f np.shape.__wrapped__ test()

In [151]:
|"a @ b

SyntaxError: unterminated string literal (detected at line 1) (3271358387.py, line 1)

In [ ]:
%lprun -f na.AbstractVectorArray.cartesian_nd.fget a.cartesian_nd

In [3]:
na.broadcast_shapes((dict(x=2, y=3), dict(y=3, z=4)))

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at C:\Users\byrdie\Kankelborg-Group\named_arrays\named_arrays\_core.py (166)[0m
[1m
File "..\..\..\..\..\..\..\named_arrays\named_arrays\_core.py", line 166:[0m
[1mdef type_array(
    <source elided>

[1m@numba.njit
[0m[1m^[0m[0m 

This error may have been caused by the following argument(s):
- argument 0: [1mCannot determine Numba type of <class 'tuple'>[0m


In [4]:
import numba

@numba.njit
def foo(*args: dict[str, int]):
    for a in args:
        print(a)

In [14]:
foo(dd, dd)

{x: 2}
{x: 2}


In [13]:
dd = numba.typed.Dict()
dd["x"] = 2
dd


DictType[unicode_type,int64]<iv=None>({x: 2})

In [19]:
p = np.random.uniform(0, 1, (11, 12, 13))

In [20]:
%timeit np.shape(p)

233 ns ± 13.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [21]:
%timeit p.shape

49.5 ns ± 0.168 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [121]:
%timeit isinstance(a, na.AbstractArray)

177 ns ± 1.92 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [122]:
%timeit hasattr(a, "__named_array_function__")

52.7 ns ± 1.18 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [172]:
import astropy.units as u
import astropy.coordinates

In [175]:
a = astropy.coordinates.SkyCoord(
    ra=na.ScalarArray([1, 2, 3] * u.deg, axes="x"),
    dec=na.ScalarArray([4, 5, 6] * u.deg, axes="x"),
    frame="icrs",
)

ValueError: item not supported by array with type <class 'named_arrays._scalars.scalars.ScalarArray'>

In [178]:
%timeit a = ()

9.97 ns ± 0.0167 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [179]:
%timeit a = tuple()

17.5 ns ± 0.0237 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [180]:
type(())

tuple

In [189]:
v = np.random.uniform(0, 1, (11, 11))
%timeit a = na.ScalarArray(v, axes=("x", "y"))

537 ns ± 9.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [204]:
a = na.NormalUncertainScalarArray(na.linspace(0, 1, axis="x", num=10), width=1, num_distribution=11)

In [207]:
a.explicit

UncertainScalarArray(
    nominal=ScalarArray(
        ndarray=[0.        , 0.11111111, 0.22222222, 0.33333333,
                 0.44444444, 0.55555556, 0.66666667, 0.77777778,
                 0.88888889, 1.        ],
        axes=('x',),
    ),
    distribution=ScalarNormalRandomSample(center=ScalarArray(
    ndarray=[0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
             0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ],
    axes=('x',),
), width=1, shape_random={'_distribution': 11}, seed=271658078407812919252693287150346625703),
)

In [208]:
a.shape_distribution

{'x': 10, '_distribution': 11}

In [209]:
a = na.UncertainScalarArray(
    nominal=na.ScalarArray(
        ndarray=np.array([ True,  True]),
        axes=('y',),
    ),
    distribution=na.ScalarArray(
        ndarray=np.array([[ True],
                 [ True]]),
        axes=('y', '_distribution'),
    ),
)

In [210]:
a.shape

{'y': 2}